# Lab 9 - Detección de Anomalías con Autoencoder, Isolation Forest y LOF

In [ ]:
from sklearn.datasets import fetch_covtype
import numpy as np 
import pandas as pd


In [ ]:
bunch = fetch_covtype(as_frame=True)

df: pd.DataFrame = bunch.data 
srs: pd.Series = bunch.target

print(df.shape, srs.shape)